In [ ]:
from glob import glob
import os
import itertools
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from mpl_toolkits.axes_grid1 import ImageGrid, make_axes_locatable, host_subplot
import matplotlib.ticker as ticker
from matplotlib.collections import Collection, LineCollection
from matplotlib.artist import allow_rasterization
from matplotlib.font_manager import FontProperties
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.dates as md
import matplotlib.ticker as ticker
from metpy.plots import ctables
#from pyPIPS import disdrometer_module as dis
from pyPIPS import pips_io as pipsio
from pyPIPS import timemodule as tm
from pyPIPS.utils import mtokm, getTimeWindow, interp_along_1D
from pyPIPS import plotmodule as pm
from pyPIPS import PIPS as pips
from pyPIPS import fmcw
from pyPIPS import DSDlib as dsd
from itertools import cycle
import netCDF4 as netcdf
from datetime import datetime, timedelta
import pandas as pd
import xarray as xr
from skimage.restoration import unwrap_phase
import skimage.feature as SF
import numpy.ma as ma
from scipy import ndimage, interpolate
%load_ext autoreload
%autoreload 2
%matplotlib inline
# plt.rcParams["figure.figsize"] = (20,10)
# fmcw.plot_fmcw_4panel_xarray(fmcw_merged_dataset)

In [ ]:
fmcw_data_base_dir = '/Volumes/scr_fast/Projects/VORTEXSE/obsdata/2017/FMCW'

# Uncomment below for March 27th, 2017 case
date = '0327'
starttimestring = '201703271900'
endtimestring = '201703272200'

# Uncomment below for April 30th, 2017 case
# date = '0430'
# starttimestring = '201704302000'
# endtimestring = '201704302359

# Create datetime objects for start and end times
datetime_start = datetime.strptime(starttimestring, '%Y%m%d%H%M%S')
datetime_end = datetime.strptime(endtimestring, '%Y%m%d%H%M%S')

# Create strings for start and end times formatted differently
starttimestring2 = datetime_start.strftime('%Y-%m-%dT%H:%M')
endtimestring2 = datetime_end.strftime('%Y-%m-%dT%H:%M')

fmcw_data_dir = os.path.join(fmcw_data_base_dir, date)

# Read in previously merged, dealiased, and corrected (for reflectivity bias) FMCW dataset
fmcw_filename = 'fmcw_merged_corrected_resampled_{}_{}.nc'.format(starttimestring, endtimestring)
fmcw_filepath = os.path.join(fmcw_data_dir, fmcw_filename)
fmcw_dataset = xr.open_dataset(fmcw_filepath)

print(fmcw_dataset)

In [ ]:
# Read in previously subsetted (to between start and end times) PIPS data
PIPS_data_dir = '/Volumes/scr_fast/Projects/VORTEXSE/obsdata/full_PIPS_dataset_RB15/subsets'
hour_min_start_string = datetime_start.strftime('%H%M')
hour_min_end_string = datetime_end.strftime('%H%M')
PIPS_filename = 'parsivel_combined_FMCW_2017_{}17_PIPS2A_60s_{}-{}.nc'.format(date, hour_min_start_string,
                                                                              hour_min_end_string)
PIPS_filepath = os.path.join(PIPS_data_dir, PIPS_filename)
PIPS_ds = xr.load_dataset(PIPS_filepath)
print(PIPS_ds)

In [ ]:
# # Resample FMCW dataset to PIPS times
# fmcw_resampled_ds = fmcw_dataset.resample(time='10S').interpolate()
# print(fmcw_resampled_ds)
# fmcw_ds_rolling = fmcw_resampled_ds.rolling(time=6)
# fmcw_resampled_ds = fmcw_ds_rolling.mean()
# print(fmcw_resampled_ds)

In [ ]:
# fmcw_resampled_at_PIPS_ds = fmcw_resampled_ds.interp_like(PIPS_ds)

In [ ]:
# fmcw_resampled_at_PIPS_ds.attrs = fmcw_dataset.attrs
# print(fmcw_resampled_at_PIPS_ds)
# fmcw.plot_fmcw_4panel_xarray(fmcw_resampled_at_PIPS_ds)

In [ ]:
# fmcw_out_filename = 'fmcw_merged_corrected_resampled_{}_{}.nc'.format(starttimestring, endtimestring)
# fmcw_filepath = os.path.join(fmcw_data_dir, fmcw_out_filename)
# fmcw_resampled_at_PIPS_ds.to_netcdf(fmcw_filepath)

In [ ]:
import pyPIPS.parsivel_params as pp
import pyPIPS.polarimetric as dp
min_diameter = pp.parsivel_parameters['min_diameter_bins_mm']
max_diameter = pp.parsivel_parameters['max_diameter_bins_mm']
bin_width = max_diameter - min_diameter
avg_diameter = pp.parsivel_parameters['avg_diameter_bins_mm']
min_fall_bins = pp.parsivel_parameters['min_fallspeed_bins_mps']
max_fall_bins = pp.parsivel_parameters['max_fallspeed_bins_mps']
avg_fall_bins = pp.parsivel_parameters['avg_fallspeed_bins_mps']

vd_matrix_da = PIPS_ds['VD_matrix_qc']
ND = PIPS_ds['ND_qc']
logND = np.log10(ND)

# conv_df = pips.calc_thermo(conv_df)
# # empirical_fallspeed = pips.calc_empirical_fallspeed(avg_diameter)
# fallspeed_spectrum = pips.calc_fallspeed_spectrum(avg_diameter, avg_fall_bins, correct_rho=True,
#                                                   rho=conv_df['rho'])
# vd_matrix_da = vd_matrix_da.where(vd_matrix_da > 0.0)
# ND = pips.calc_ND(vd_matrix_da, fallspeed_spectrum, 10.)
# ND = ND.T
# logND = np.log10(ND)
# logND = 

PSD_datetimes = pips.get_PSD_datetimes(vd_matrix_da)
PSD_datetimes_dict = pips.get_PSD_time_bins(PSD_datetimes)

PSD_edgetimes = md.date2num(PSD_datetimes_dict['PSD_datetimes_edges'])
PSD_centertimes = md.date2num(PSD_datetimes_dict['PSD_datetimes_centers'])

# ND = PIPS_dict['ND'].T
# logND = ma.log10(ND)
# DSD_index = PIPS_dict['DSD_index']
# DSD_interval = PIPS_dict['DSD_interval']
# DSD_interval_td = timedelta(seconds=DSD_interval)
# DSD_halfinterval_td = timedelta(seconds=DSD_interval / 2.)

# Determine start and end times/indices for analysis
# PSDtimestamps = PIPS_dict['PSDtimestamps']
# PSDtimestampsnums = md.date2num(PSDtimestamps)

#     plotstarttime = starttime
#     plotstoptime = stoptime

# pstartindex, pstopindex = getTimeWindow(starttimestring, endtimestring, PSDtimestampsnums)

# pstarttime = PSDtimestampsnums[pstartindex]
# pstoptime = PSDtimestampsnums[pstopindex]

# plotstarttime = pstarttime
# plotstoptime = pstoptime

# PSDtimestamps_edge = [x - DSD_interval_td for x in PSDtimestamps]
# # Add an extra 10 sec for the last time bin boundary
# PSDtimestamps_edge.append(PSDtimestamps_edge[-1] + DSD_interval_td)
# PSDtimestamps_avg = [x - DSD_halfinterval_td for x in PSDtimestamps]
# PSDstarttimes = md.date2num(PSDtimestamps_edge[pstartindex:pstopindex + 1])
# PSDmidtimes = md.date2num(PSDtimestamps_avg[pstartindex:pstopindex + 1])

scattfile = '/Users/dawson29/Projects/pyPIPS/tmatrix/FMCW/SCTT_RAIN_fw100.dat'
# Observed DSD: TODO, need to make ND, ND_expDSD, and ND_gamDSD have consistent units!
dualpol_dis = dp.calpolrain(15.0, scattfile, ND, bin_width)
#print(dualpol_dis)
dis_ref = dualpol_dis.get('REF', np.empty((0)))
logND_plot = logND.T
disvars = {'min_diameter': min_diameter, 'PSDstarttimes': PSD_edgetimes,
           'PSDmidtimes': PSD_centertimes, 'logND': logND_plot}
disvars['REF'] = dis_ref

# Quick and dirty rayleigh scattering approximation

M6 = []
print(ND.shape)
for t in range(ND.shape[0]):
    temp_M6 = ((avg_diameter[:] / 1000.)**6.) * (1000. * ND[t, :]) * bin_width[:] / 1000.
    temp_M6 = np.nansum(temp_M6)
    M6.append(temp_M6)

M6 = np.array(M6, dtype=np.float64)
refl_disd = 10.0 * np.log10(1e18 * M6)
#disvars['dBZ'] = refl_disd

# fig, ax = plt.subplots(figsize=(8, 3))

# ax.plot(PSD_centertimes, disvars['dBZ'])

radz = 500. # (m) Height of FMCW gate to plot
radzindex = int(radz/5.)

plotx_rad = fmcw_dataset['time'].values
#dBZ_D_plt = fmcw_merged_dataset['Ze'].values[:, radzindex] + 10.**(2.*7.3/10.)
dBZ_D_plt = fmcw_dataset['Ze_corr'].values[:, radzindex]


# print(fmcw_merged_dataset['height'][radzindex])
# print(fmcw_merged_dataset['Ze_corr'][:, radzindex])

# fmcw_merged_dataset['Ze'][:, radzindex].plot()

# print(dBZ_D_plt)

radvars = {'radmidtimes': plotx_rad, 'REF': dBZ_D_plt}

# Prepare axis parameters
# timelimits = [md.date2num(datetime.strptime(starttimestring, tm.timefmt3)),
#               md.date2num(datetime.strptime(endtimestring, tm.timefmt3))]
# timelimits = [md.date2num(datetime.strptime('20170430203000', tm.timefmt3)),
#               md.date2num(datetime.strptime('20170430210000', tm.timefmt3))]
# Set up axis parameters
start_time = '20170327200000'
end_time = '20170327204500'
try:
    start_datetime = datetime.strptime(start_time, tm.timefmt3)
    print('start_datetime', start_datetime)
except (ValueError, TypeError):
    start_datetime = PSD_edgetimes[0]
try:
    end_datetime = datetime.strptime(end_time, tm.timefmt3)
    print('end_datetime', end_datetime)
except (ValueError, TypeError):
    end_datetime = PSD_edgetimes[-1]
timelimits = [start_datetime, end_datetime]
diamlimits = [0.0, 9.0]
diamytick = 1.0

locator = md.MinuteLocator(byminute=np.arange(0., 65., 5.))# md.HourLocator(interval=1)
minorlocator = md.MinuteLocator(byminute=np.arange(0., 61., 1.))
# minorlocator = md.HourLocator(interval=1)
#dateformat = '%d/%H'
dateformat = '%H:%M'
formatter = md.DateFormatter(dateformat)
axparams = {'majorxlocator': locator, 'majorxformatter': formatter,
            'minorxlocator': minorlocator,
            'axeslimits': [timelimits, diamlimits],
            'majorylocator': ticker.MultipleLocator(base=diamytick),
            'axeslabels': [None, 'D (mm)']}

            # Ok, now we should have everything ready to go to plot the meteograms.
            # Let'er rip!
meteogram_image_dir = './'
dis_plot_name = 'PIPS2A_observed'
pm.plotDSDmeteograms(dis_plot_name, meteogram_image_dir,
                     axparams, disvars, radvars.copy(), close_fig=False)

In [ ]:
dBZ_D_plt = fmcw_dataset['Ze_corr'].sel(time=slice('2017-03-27T20:00', '2017-03-27T20:45'))
dBZ_D_plt.sel(height=[500., 750., 1000., 1250., 1500.]).plot.line(x='time')

In [ ]:
fmcw_dataset_2000_2045 = fmcw_dataset.sel(
    time=slice('2017-03-27T20:00', '2017-03-27T20:45'))

print(fmcw_dataset_2000_2045)

fig, ax = plt.subplots(figsize=(10, 10))

# Reflectivity (Ze)
#     m1 = ax[0, 0].pcolormesh(UTCtime, height/1000., Ze.T,cmap=plasma_white, \
#                              vmin=-30., vmax=30.)

m1 = fmcw_dataset_2000_2045['Ze_corr'].plot(x='time', y='height', ax=ax, 
                                            cmap=fmcw.plasma_white, vmin=-20., vmax=60.)

ax.set_title("(a) " + fmcw_dataset_2000_2045['Zef_corr'].name + " (dBZ)")
ax.set_ylim(fmcw_dataset_2000_2045['height'].min(), 
            fmcw_dataset_2000_2045['height'].max())
# ax[0, 0].xaxis.set_major_locator(md.MinuteLocator(byminute=range(0, 60, 10)))
# ax[0, 0].xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.yaxis.set_major_formatter(ticker.FuncFormatter(mtokm))
#plt.xticks(rotation=30)
#ax[0, 0].set_xlabel("Time (UTC)")
ax.set_ylabel("Height (km AGL)")
# plt.colorbar(m1,ax=ax[0, 0])

#     # Dealiased Doppler (vertical) velocity (vel_da)

#     m2 = dataset['vel_da'].plot(x='time', y='height', ax=ax[0, 1], cmap=cm.RdBu_r,
#                                 vmin=-2.*np.ceil(dataset.Vmax), vmax=2.*np.ceil(dataset.Vmax))

# #     m2 = ax[0, 1].pcolormesh(UTCtime,height/1000., vel.T, cmap=cm.RdBu_r, \
# #                              vmin=-1.*np.ceil(Vmax), vmax=np.ceil(Vmax))

#     ax[0, 1].set_title("(b) " + dataset['vel_da'].name + r" (m s$^{-1}$)")
#     ax[0, 1].set_ylim(dataset['height'].min(), dataset['height'].max())
#     # ax[0, 1].xaxis.set_major_locator(md.MinuteLocator(byminute=range(0, 60, 10)))
#     # ax[0, 1].xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
#     ax[0, 1].yaxis.set_major_formatter(ticker.FuncFormatter(mtokm))
#     #plt.xticks(rotation=30)
#     #ax[0, 1].set_xlabel("Time (UTC)")
#     #ax[0, 1].set_ylabel("Height (km AGL)")
#     # plt.colorbar(m2,ax=ax[0, 1])

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

m1 = fmcw_dataset_2000_2045['Ze_corr'].plot(x='time', y='height', ax=ax, 
                                       cmap=fmcw.plasma_white, vmin=-20., vmax=60.)

ax.set_title("(a) " + fmcw_dataset_2000_2045['Ze_corr'].name + " (dBZ)")
ax.set_ylim(fmcw_dataset_2000_2045['height'].min(), 
            fmcw_dataset_2000_2045['height'].max())
# ax[0, 0].xaxis.set_major_locator(md.MinuteLocator(byminute=range(0, 60, 10)))
# ax[0, 0].xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.yaxis.set_major_formatter(ticker.FuncFormatter(mtokm))
#plt.xticks(rotation=30)
#ax[0, 0].set_xlabel("Time (UTC)")
ax.set_ylabel("Height (km AGL)")


fig, ax = plt.subplots(figsize=(10, 10))

Zef = fmcw_dataset_2000_2045['Zef_corr']
Zef_lin = 10.**(Zef / 10.)
Ze_lin = Zef_lin.sum(dim='vels') # * 2. * fmcw_dataset.Vmax
Ze2 = 10. * np.log10(Ze_lin)

m1 = Ze2.plot(x='time', y='height', ax=ax, cmap=fmcw.plasma_white, vmin=-20., vmax=60.)

ax.set_title("(a) " + fmcw_dataset_2000_2045['Ze_corr'].name + " (derived, dBZ)")
ax.set_ylim(fmcw_dataset_2000_2045['height'].min(), 
            fmcw_dataset_2000_2045['height'].max())
# ax[0, 0].xaxis.set_major_locator(md.MinuteLocator(byminute=range(0, 60, 10)))
# ax[0, 0].xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.yaxis.set_major_formatter(ticker.FuncFormatter(mtokm))
#plt.xticks(rotation=30)
#ax[0, 0].set_xlabel("Time (UTC)")
ax.set_ylabel("Height (km AGL)")



In [ ]:
vbins = fmcw_dataset['vels']
vbins_dealias = vbins - vbins[-1]
fmcw_dataset.coords['vels_dealiased'] = ('vels', vbins_dealias)
#print(fmcw_merged_resampled_at_PIPS_ds)
Zef_corr = fmcw_dataset['Zef_corr']
Zef = fmcw_dataset['Zef']
time_to_plot = '2017-03-27T20:19'
#print(Zef_corr)
#print(Zef_corr.loc['2017-04-30T22:30':'2017-04-30T22:31'])
Zef_corr_onetime = Zef_corr.sel(time=time_to_plot)
Zef_onetime = Zef.sel(time=time_to_plot)
# print(Zef_corr_2230)
Zef_corr_onetime = Zef_corr_onetime.roll(vels=128, roll_coords=False)
Zef_onetime = Zef_onetime.roll(vels=128, roll_coords=False)
#delta_v = Zef_corr_onetime.vels[1] - Zef_corr_onetime.vels[0]
#print(2. / delta_v)
#shift = int(2. / delta_v)
#print(shift)
#Zef_corr_onetime = Zef_corr_onetime.roll(vels=shift, roll_coords=False)
height_to_plot = 500.
Zef_corr_onetime_oneheight = Zef_corr_onetime.sel(height=height_to_plot)
Zef_onetime_oneheight = Zef_onetime.sel(height=height_to_plot)
fig, ax = plt.subplots(figsize=(10, 10))
Zef_corr_onetime.plot(x='vels_dealiased', y='height', vmin=-20, vmax=40, ax=ax)
ax.set_ylim(0., 2500.)
#fig2, ax2 = plt.subplots()
#Zef_corr_2230_shifted.plot(ax=ax2, vmin=-30, vmax=30)
fig, ax = plt.subplots()
Zef_corr_onetime_oneheight.plot(ax=ax, x='vels_dealiased')

#Zef_corr_2230.plot(vmin=0., vmax=40.)

In [ ]:
#print(Zef_corr_onetime_oneheight)
# Convert to linear units
# Zef_onetime_oneheight = Zef_onetime_oneheight.where(Zef_onetime_oneheight > -99.)
Zef_corr_onetime_oneheight.plot(x='vels_dealiased')
Zef_lin = 10.**(Zef_corr_onetime_oneheight / 10.)
Ze_lin = Zef_lin.sum(dim='vels')
Ze = 10. * np.log10(Ze_lin)
print(Ze)
print(fmcw_dataset['Ze_corr'].sel(time=time_to_plot, height=height))



In [ ]:
# Working on forward operator here.

print(ND.sel(time=time_to_plot))
D = ND['diameter'] / 1000.
M6, ZD = dsd.calc_moment_bin(ND, moment=6)
dBZ = 10. * np.log10(1.e18 * M6.sel(time=time_to_plot))
print("dBZ = ", dBZ)
dBZD = 10. * np.log10(1.e18 * ZD)
ZD_onetime = ZD.sel(time=time_to_plot)
dBZD_onetime = dBZD.sel(time=time_to_plot)
fig, ax = plt.subplots(figsize=(10, 10))
dBZD_onetime.plot(x='diameter', ax=ax)
ax.set_xlim(0., 9.)

# Calculate terminal fall speeds per diameter bin
Vt = -pips.calc_empirical_fallspeed(ND['diameter'])

# TODO: rebin N(D), Z(D), etc. into regularly spaced velocity bins. Then can resample to FMCW velocity bins
dBZD_onetime.coords['terminal_velocity'] = ('diameter_bin', Vt)
fig2, ax2 = plt.subplots(figsize=(10,10))
dBZD_onetime.plot(x='terminal_velocity', ax=ax2)

In [ ]:
fmcw_vels = Zef_corr_onetime_oneheight['vels_dealiased'].values
fmcw_vel_bin_interval = fmcw_vels[1] - fmcw_vels[0]
# Scale ZD by velocity bin widths from parsivel terminal velocities
Vt_min = -pips.calc_empirical_fallspeed(ND['min_diameter'])
Vt_max = -pips.calc_empirical_fallspeed(ND['max_diameter'])

ZD.coords['vels'] = ('diameter_bin', Vt)
ZD.coords['min_vels'] = ('diameter_bin', Vt_min)
ZD.coords['max_vels'] = ('diameter_bin', Vt_max)

Vt = ZD['vels']
Vt_min = ZD['min_vels']
Vt_max = ZD['max_vels']
# print(Vt_min)
# print(Vt)
# print(Vt_max)
p_vel_bin_width = np.abs(Vt_max - Vt_min)
# print(p_vel_bin_width)

ZD_scaled = ZD / p_vel_bin_width
ZD_scaled = ZD_scaled.swap_dims({'diameter_bin': 'vel_bin'})
ZD_scaled = ZD_scaled.set_index(vel_bin='min_vels')
ZD_scaled_rebinned = ZD_scaled.reindex({'vel_bin': fmcw_vels}, method='pad')
ZD_rebinned = ZD_scaled_rebinned * fmcw_vel_bin_interval
# ZD_rebinned = ZD_rebinned.swap_dims({'diameter_bin': 'vel_bin'})
dBZD = 10. * np.log10(1.e18 * ZD_rebinned)
dBZD = dBZD.where(np.isfinite(dBZD))
dBZD_onetime = dBZD.sel(time=time_to_plot)
fig, ax = plt.subplots(figsize=(10, 10))
dBZD_onetime.plot(x='vel_bin', ax=ax)
Zef_corr_onetime_oneheight.plot(x='vels_dealiased', ax=ax)
ax.set_xlim(-14., 0.)
#print(dBZD_onetime)
ZD_sum = ZD_rebinned.sum(dim='vel_bin')
dBZ_sum = 10. * np.log10(1.e18 * ZD_sum)
#print(dBZ_sum.sel(time=time_to_plot))

In [ ]:
time_to_plot = '2017-03-27T20:11'
height_to_plot = 500.
sn = fmcw_dataset_2000_2045['sn'] / 10.
sn_onetime = sn.sel(time=time_to_plot)
snr = fmcw_dataset_2000_2045['snr']
sff = fmcw_dataset_2000_2045['sff'] / 10.
sff_onetime = sff.sel(time=time_to_plot)
sf = fmcw_dataset_2000_2045['sf'] / 10.
sf_onetime = sf.sel(time=time_to_plot)
Zef_onetime = fmcw_dataset_2000_2045['Zef'].sel(time=time_to_plot)

fig1, ax1 = plt.subplots(figsize=(10, 10))
Zef_onetime.plot(x='vels', ax=ax1, vmin=-20, vmax=50)

fig2, ax2 = plt.subplots(figsize=(10, 10))
Zef_onetime_oneheight = Zef_onetime.sel(height=height_to_plot)
Zef_onetime_oneheight = Zef_onetime_oneheight.where(Zef_onetime_oneheight > -99.)
# Zef_onetime_oneheight.plot(x='vels', ax=ax2)
# sff_onetime.plot(ax=ax2, vmin=-20, vmax=50)
snr.plot(ax=ax2, x='time')

In [ ]:
# Let's make a ridge plot
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="white", rc={"axes.facecolor": (0, 0, 0, 0)})

# Create the data
rs = np.random.RandomState(1979)
x = rs.randn(500)
g = np.tile(list("ABCDEFGHIJ"), 50)
df = pd.DataFrame(dict(x=x, g=g))
m = df.g.map(ord)
df["x"] += m

print(df)


Zef_corr_onetime = Zef_corr_onetime.thin({'height': 100})

Zef_corr_onetime_df = Zef_corr_onetime.to_dataframe().reset_index()
print(Zef_corr_onetime_df)
# Initialize the FacetGrid object
pal = sns.cubehelix_palette(10, rot=-.25, light=.7)
g = sns.FacetGrid(Zef_corr_onetime_df, row="height", hue='height', aspect=2, height=1., palette=pal)

# Draw the densities in a few steps
#g.map(sns.kdeplot, "vels_dealiased", clip_on=False, shade=True, alpha=1, lw=1.5, bw=.2)
#g.map(sns.kdeplot, "vels_dealiased", clip_on=False, color="w", lw=2, bw=.2)
g.map(sns.lineplot, "vels_dealiased", "Zef_corr", clip_on=False, alpha=1, lw=1.5)
#g.map(sns.lineplot, "vels_dealiased", "Zef_corr", clip_on=False, color="w", lw=2, bw=.2)
#g.map(plt.axhline, y=0, lw=2, clip_on=False)


# Define and use a simple function to label the plot in axes coordinates
def label(x, color, label):
    ax = plt.gca()
    ax.text(0, .2, label, fontweight="bold", color=color,
            ha="left", va="center", transform=ax.transAxes)


g.map(label, "vels_dealiased")

# Set the subplots to overlap
# g.fig.subplots_adjust(hspace=-.25)

# Remove axes details that don't play well with overlap
g.set_titles("")
#g.set(yticks=[])
#g.despine(bottom=True, left=True)

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="white", rc={"axes.facecolor": (0, 0, 0, 0)})

# Create the data
rs = np.random.RandomState(1979)
x = rs.randn(500)
g = np.tile(list("ABCDEFGHIJ"), 50)
df = pd.DataFrame(dict(x=x, g=g))
m = df.g.map(ord)
df["x"] += m
print(df)

# Initialize the FacetGrid object
pal = sns.cubehelix_palette(10, rot=-.25, light=.7)
g = sns.FacetGrid(df, row="g", hue="g", aspect=15, height=.5, palette=pal)

# Draw the densities in a few steps
g.map(sns.kdeplot, "x", clip_on=False, shade=True, alpha=1, lw=1.5, bw=.2)
g.map(sns.kdeplot, "x", clip_on=False, color="w", lw=2, bw=.2)
g.map(plt.axhline, y=0, lw=2, clip_on=False)


# Define and use a simple function to label the plot in axes coordinates
def label(x, color, label):
    ax = plt.gca()
    ax.text(0, .2, label, fontweight="bold", color=color,
            ha="left", va="center", transform=ax.transAxes)


g.map(label, "x")

# Set the subplots to overlap
g.fig.subplots_adjust(hspace=-.25)

# Remove axes details that don't play well with overlap
g.set_titles("")
g.set(yticks=[])
g.despine(bottom=True, left=True)